In [24]:
from Models import ML_Models
exam_models = ML_Models()

In [25]:
import yfinance as yf
import pandas as pd
import inspect
import datetime as dt
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
# Parameters used for fetching the data from yfinance.
start_date = "2012-01-01"
end_date = dt.date.today()
main_col = "Adj Close"
interval = "1d"
stocks_list = ["EQNR.OL", "DNB.OL", "TEL.OL", "NHY.OL", "AKRBP.OL", "YAR.OL", "MOWI.OL", "CL=F", "OSEBX.OL"]

# Specifying the indicators wanted for further analysis.
indicators = ["MA5", "MA20", "MA50", "MA200", "MIN", "MAX", "LOG_RET", "MOM", "VOLA", "DIFF"]

# Models to utilize for forecasting/prediction.
models = ["LR", "DTR", "MLP", "XGBoost", "XGBoost_LR", "ADA", "GBR", "Bagging", "StackedRegressor"]

# Metrics used to evaluate the performance of each model.
# MAE, MSE, RMSE and MAPE are named with "neg_" to be recognized by the cross_validate function from scikit-learn.
metric_names = ["r2", "neg_mean_absolute_error", "neg_mean_squared_error", "neg_root_mean_squared_error", "neg_mean_absolute_percentage_error"]
pretty_metric_names = {"r2":"R^2: ", "neg_mean_absolute_error":"MAE: ", "neg_mean_squared_error":"MSE: ","neg_root_mean_squared_error":"RMSE: ", "neg_mean_absolute_percentage_error":"MAPE: "}

# Collecting data from Yahoo Finance

In [27]:
stock_data = {}
for ticker in stocks_list:
    print(f"Downloading {ticker} data")
    # fetch stock data from yahoo finance
    raw_data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    stock_data[ticker] = raw_data

print("All the data is now downloaded!")

# Save fetched data to csv
for ticker in stocks_list:
    #stock_data[ticker].to_csv("raw_data/data_"+ticker+".csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
All the data is now downloaded!


# Preproccesing the data

In [28]:
def add_indicator_columns(data, indicators):
    # Creating label and shifting the selected main_col value by 1.
    label_name = "Label"
    data[label_name] = data[main_col].shift(periods=1)

    # Checking which of the different indicators that should be added as a column (based on input from indicators list).
    if "MA5" in indicators:
        data["MA5"] = data[label_name].rolling(5).mean()
    if "MA20" in indicators:
        data["MA20"] = data[label_name].rolling(20).mean()
    if "MA50" in indicators:
        data["MA50"] = data[label_name].rolling(50).mean()
    if "MA200" in indicators:
        data["MA200"] = data[label_name].rolling(200).mean()
    if "MIN" in indicators:
        data["MIN"] = data[label_name].rolling(20).min()
    if "MAX" in indicators:
        data["MAX"] = data[label_name].rolling(20).max()
    log_ret = np.log(data[label_name] / data[label_name].shift(1))
    if "LOG_RET" in indicators:
        data["LOG_RET"] = log_ret
    if "MOM" in indicators:
        data["MOM"] = log_ret.rolling(20).mean()
    if "VOLA" in indicators:
        data["VOLA"] = log_ret.rolling(20).std()
    if "DIFF" in indicators:
        data["DIFF"] = data[label_name] - data[label_name].shift(1)

    # remove empty vals.
    data.dropna(axis=0, inplace=True)

In [29]:
def create_X_y_arrays(data, label_name):
        # array that contains the indicators data.
        X = data.loc[:, indicators].to_numpy()
        # array with the target data (based on main_col).
        y = data[label_name].to_numpy()
        return X, y

In [30]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

def create_X_y_train_test_split(X, y, current_stock):
    data = stock_data[current_stock]

    for train_index, test_index in tscv.split(data):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test

In [31]:
# Adding the specified indicators to the data.
for ticker, data in stock_data.items():
    add_indicator_columns(data=data, indicators=indicators)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Training models

In [32]:
cv_results = {}
trained_models = {}

def train_models(input_models):
        for ticker, data in stock_data.items():

            X, y = create_X_y_arrays(data=data, label_name="Label")

            X_train, X_test, y_train, y_test = create_X_y_train_test_split(X=X, y=y, current_stock=ticker)

            # Evaluating and training selected models.
            for model_i in input_models:
                model = exam_models.pick_model(model=model_i)
                metric_dict = {}
                for metric_name in metric_names:
                    metric_dict[metric_name] = metric_name
                    
                # using method from sci-kit lib to cross-validate
                cross_val_results = cross_validate(
                    model,
                    X,
                    y,
                    cv=tscv,
                    scoring=metric_dict,
                    return_train_score=True,
                    n_jobs=-1,
                    verbose=0  
                )
                model.fit(X_train, y_train)
                cv_results[ticker+"_model_"+model_i] = cross_val_results 
                trained_models["trained_model_"+model_i+"_"+ticker] = model

        return cv_results

In [33]:
cv_stocks_models = train_models(input_models=models)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

# Predicting values based on trained models 

In [34]:
stock_predictions = {}
def predict_trained_models(input_models):
    for ticker, data in stock_data.items():

        # Creating X and y arrays for train and test sets.
        X, y = create_X_y_arrays(data=data, label_name= "Label")

        X_train, X_test, y_train, y_test = create_X_y_train_test_split(X=X, y=y, current_stock=ticker)

        last_train_index, last_test_index = None, None

        for train_index, test_index in tscv.split(data):
            last_train_index, last_test_index = train_index, test_index

        prediction = data.loc[data.index[last_test_index], [main_col]].copy(deep=True)
        stock_predictions[ticker] = prediction

        for model_i in input_models:
            model = trained_models["trained_model_"+model_i+"_"+ticker]
            y_pred = model.predict(X_test)
            prediction.loc[:, model_i+" Prediction"] = y_pred

    return stock_predictions

In [35]:
predicted_stock_data = predict_trained_models(input_models=models)

In [36]:
for ticker in stocks_list:
    print(ticker)
    display(predicted_stock_data[ticker])

EQNR.OL


,Adj Close,LR Prediction,DTR Prediction,MLP Prediction,XGBoost Prediction,XGBoost_LR Prediction,ADA Prediction,GBR Prediction,Bagging Prediction,StackedRegressor Prediction
Date,,,,,,,,,,
2021-08-11,177.834885,172.316810,172.243942,172.558017,173.507355,171.043854,172.470428,173.267839,173.068433,172.328178
2021-08-12,180.592270,176.806584,175.946594,176.924242,175.033615,174.888245,173.154510,176.667954,174.750446,176.703754
2021-08-13,181.174835,179.059650,175.822571,178.393636,179.794601,177.517609,175.524471,178.579158,179.634190,179.073726
2021-08-16,178.902863,178.961513,175.822571,178.197804,179.922943,177.458664,175.524471,178.967578,181.096266,178.954194
2021-08-17,183.330307,178.637226,181.806335,177.326883,181.090668,177.455292,177.672146,178.963687,179.980495,178.631437
...,...,...,...,...,...,...,...,...,...,...
2023-05-02,297.100006,306.968082,189.201675,310.413344,194.206619,292.572083,191.309291,197.213052,192.543054,300.526632
2023-05-03,293.399994,295.793690,188.859863,297.858180,190.254135,283.107758,190.840377,191.138348,190.492087,289.607508
2023-05-04,296.149994,295.854591,188.859863,299.263676,189.804108,283.386871,190.840377,191.291655,190.645909,289.692049


DNB.OL


,Adj Close,LR Prediction,DTR Prediction,MLP Prediction,XGBoost Prediction,XGBoost_LR Prediction,ADA Prediction,GBR Prediction,Bagging Prediction,StackedRegressor Prediction
Date,,,,,,,,,,
2021-06-24,165.566391,165.328389,160.323730,164.723173,159.140671,166.024445,152.813074,159.964588,159.194864,165.298703
2021-06-25,167.595779,163.058997,160.323730,162.243159,159.400848,163.052368,152.813074,159.157052,159.296333,163.043700
2021-06-28,163.832916,166.527248,160.323730,165.724866,159.256241,165.322906,152.813074,159.014374,159.194864,166.547620
2021-06-29,162.353134,163.436365,154.954224,162.098372,156.675751,161.854477,152.813074,156.363562,157.943385,163.368363
2021-06-30,158.632553,165.053061,157.829224,164.243310,157.786758,162.847305,152.813074,158.157395,157.638971,165.039035
...,...,...,...,...,...,...,...,...,...,...
2023-05-02,187.000000,186.236150,158.970795,186.429129,157.936050,187.960373,152.813074,159.504216,157.638983,186.389280
2023-05-03,187.750000,184.115197,159.393570,183.943011,158.275589,186.103012,152.813074,158.501943,157.182356,184.303785
2023-05-04,183.149994,186.713504,158.463409,187.138675,158.029526,187.151611,152.813074,158.703219,157.266920,186.933848


TEL.OL


...

,Adj Close,LR Prediction,DTR Prediction,MLP Prediction,XGBoost Prediction,XGBoost_LR Prediction,ADA Prediction,GBR Prediction,Bagging Prediction,StackedRegressor Prediction
Date,,,,,,,,,,
2021-08-11,215.904449,219.277143,216.575272,217.405019,217.550354,218.676453,214.635613,218.417178,217.868967,219.263354
2021-08-12,216.287766,217.267966,214.121323,215.149052,218.125473,217.685043,214.635613,217.241738,217.916881,217.168788
2021-08-13,217.054398,217.417704,217.246063,215.884530,216.535248,217.933319,214.635613,216.451197,216.517520,217.367880
2021-08-16,216.958542,217.590599,217.246063,216.133840,216.325745,218.288147,214.635613,216.690617,216.564281,217.537270
2021-08-17,216.096130,218.028584,217.246063,215.662954,215.928650,218.767090,214.635613,216.505656,216.790698,217.997600
...,...,...,...,...,...,...,...,...,...,...
2023-05-02,201.600006,201.056443,201.276337,202.359313,200.872910,201.411545,197.779186,203.195044,202.258742,201.190437
2023-05-03,199.800003,200.289406,198.502655,200.513331,202.551651,200.062714,201.090623,201.154306,200.887828,200.225112
2023-05-04,196.500000,199.756678,198.502655,200.534391,201.964355,199.107666,201.090623,201.106923,200.710956,199.724090


CL=F


,Adj Close,LR Prediction,DTR Prediction,MLP Prediction,XGBoost Prediction,XGBoost_LR Prediction,ADA Prediction,GBR Prediction,Bagging Prediction,StackedRegressor Prediction
Date,,,,,,,,,,
2021-08-13,68.440002,68.165812,66.820000,67.996134,68.740768,69.134277,69.023731,68.495539,67.295000,68.145306
2021-08-16,67.290001,67.899155,66.480003,67.649485,68.562996,68.643890,69.023731,67.955054,67.410001,67.893440
2021-08-17,66.589996,67.886667,66.430000,67.473132,67.772232,69.689308,69.023731,67.325084,67.495000,67.806822
2021-08-18,65.459999,67.789805,66.430000,67.460303,68.158363,69.411011,67.947079,67.857614,67.639999,67.733627
2021-08-19,63.689999,66.443057,66.809998,66.365724,66.044060,67.855118,67.138213,66.453508,66.535000,66.371286
...,...,...,...,...,...,...,...,...,...,...
2023-05-02,71.660004,74.620481,74.209999,75.374098,75.419594,75.149574,73.729856,74.381626,74.882999,74.830649
2023-05-03,68.599998,70.845895,74.580002,71.989872,74.956955,70.775650,73.426117,72.282750,73.865998,71.286449
2023-05-04,68.559998,69.850691,71.650002,71.508773,70.132805,69.676292,72.277657,70.915482,69.914999,69.955306


OSEBX.OL


,Adj Close,LR Prediction,DTR Prediction,MLP Prediction,XGBoost Prediction,XGBoost_LR Prediction,ADA Prediction,GBR Prediction,Bagging Prediction,StackedRegressor Prediction
Date,,,,,,,,,,
2021-10-22,1217.709961,1211.440835,1217.469971,1198.608059,1207.861328,1186.326538,1189.720101,1202.033309,1206.851978,1211.937267
2021-10-25,1215.099976,1219.210989,1217.469971,1206.956815,1209.122437,1194.847900,1189.720101,1203.789945,1207.803979,1219.897165
2021-10-26,1215.739990,1215.234722,1217.469971,1203.440291,1207.066284,1190.081909,1189.720101,1203.063291,1206.851978,1215.823763
2021-10-27,1211.869995,1218.774696,1217.469971,1208.287866,1207.745117,1195.960449,1189.720101,1203.063291,1207.803979,1219.428946
2021-10-28,1202.089966,1213.151088,1217.469971,1205.008373,1208.679077,1191.535522,1189.720101,1202.033309,1206.851978,1213.767367
...,...,...,...,...,...,...,...,...,...,...
2023-05-02,1208.410034,1221.809359,1217.469971,1221.840198,1211.428711,1212.672485,1189.720101,1208.909152,1207.803979,1222.779471
2023-05-03,1203.479980,1199.928514,1217.469971,1200.306298,1199.211182,1187.240234,1189.720101,1193.040876,1207.096985,1200.858668
2023-05-04,1189.719971,1209.028723,1217.469971,1213.750717,1204.815674,1195.505249,1189.720101,1198.345987,1205.402979,1210.470494


In [ ]:
metrics_df_output = {}
def print_metrics(save_data_to_df=True):
    for ticker, data in stock_data.items():
        print("\n--------", ticker, "--------")

        # prepare dataframe if requested
        if save_data_to_df:
            metrics_df = pd.DataFrame(index=list(pretty_metric_names.values()))
            metrics_df_output[ticker] = metrics_df
            current_metrics_df = metrics_df

        for model_name in models:        
            cv = cv_results[ticker+"_model_"+model_name]

            print("-", model_name, "-")
            print("-", "Training Scores:", "-")
            split1_errors_string = f"Score for first data split \n"
            split3_errors_string = f"Score for third data split \n"
            split5_errors_string = f"Score for last data split \n"

            for error_metric_name in metric_names:
                # some metrics are saved as negative, so change sign
                if error_metric_name.startswith("neg"):
                    try:
                        error_metric_value = -cv["train"+"_"+error_metric_name]
                    # string being passed that can't be negative
                    except:
                        error_metric_value = cv["train"+"_"+error_metric_name]
                else:
                    error_metric_value = cv["train"+"_"+error_metric_name]

                if save_data_to_df:
                    # pass string, used to indicate missing metrics
                    if isinstance(error_metric_value, str):
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Train"] = error_metric_value
                    # otherwise mean of all splits
                    else:
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Train"] = np.mean(error_metric_value)
        
                if isinstance(error_metric_value, str):
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[0]:.3f}\n"

                if isinstance(error_metric_value, str):
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[2]:.3f}\n"
    
                if isinstance(error_metric_value, str):
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[-1]:.3f}\n"

            # print the two strings
            print(split1_errors_string+"\n"+split3_errors_string+"\n"+split5_errors_string+"\n")
            
            print("-", "Testing Scores:", "-")
            split1_errors_string = f"Score for first data split \n"
            split3_errors_string = f"Score for third data split \n"
            split5_errors_string = f"Score for last data split \n"

            for error_metric_name in metric_names:
                # some metrics are saved as negative, so change sign
                if error_metric_name.startswith("neg"):
                    try:
                        error_metric_value = -cv["test"+"_"+error_metric_name]
                    # string being passed that can't be negative
                    except:
                        error_metric_value = cv["test"+"_"+error_metric_name]
                else:
                    error_metric_value = cv["test"+"_"+error_metric_name]

                if save_data_to_df:
                    # pass string, used to indicate missing metrics
                    if isinstance(error_metric_value, str):
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Test"] = error_metric_value
                    # otherwise mean of all splits
                    else:
                        current_metrics_df.loc[pretty_metric_names[error_metric_name], model_name+" Model "+"Test"] = np.mean(error_metric_value)
        
                if isinstance(error_metric_value, str):
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split1_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[0]:.3f}\n"

                if isinstance(error_metric_value, str):
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split3_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[2]:.3f}\n"

                if isinstance(error_metric_value, str):
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value}\n"
                else:
                    split5_errors_string += f"{pretty_metric_names[error_metric_name]} {error_metric_value[-1]:.3f}\n"

            # print the two strings
            print(split1_errors_string+"\n"+split3_errors_string+"\n"+split5_errors_string+"\n")

    if save_data_to_df:
        return metrics_df_output
    else:
        return None

In [ ]:
metrics_output = print_metrics(save_data_to_df=True)

# Plots for insights about the data

In [ ]:
# PLOT FOR SHOWING DIFFERENT DATA SPLITS FOR THE DIFFERENT STOCKS
for ticker in stocks_list:
    fig, sub_plots = plt.subplots(n_splits, figsize=(16,20))
    fig.set_tight_layout(True)

    data = stock_data[ticker]
    idx = data.index

    tscv = TimeSeriesSplit(n_splits=n_splits)

    splits = list(tscv.split(data))
        
    current_split = 1
    for i in range(len(sub_plots)):
        train_index, test_index = splits[i]

        sub_plots[i].plot(idx[train_index], data.loc[idx[train_index], main_col], label=f"Training data {current_split}", color="blue")
        sub_plots[i].plot(idx[test_index], data.loc[idx[test_index], main_col], label=f"Test data {current_split}", color="red")
        sub_plots[i].set_xlim(idx[0], idx[-1])
        sub_plots[i].set_title(f"Train / test split {current_split} for {ticker}")
        sub_plots[i].set_xlabel("Date")
        sub_plots[i].set_ylabel(f"{main_col}")
        sub_plots[i].legend()

        current_split = current_split + 1
    
    #fig.savefig("data_splits_plots/"+ticker+".png")

fig.show()

In [ ]:
# PLOT FOR COMPARING ACTUAL TO THE DIFFERENT MODELS PREDICTED VALUES
for ticker, data in stock_data.items():
    figure, axs = plt.subplots(figsize=(32,16))
    
    X, y = create_X_y_arrays(data=data, label_name="Label")
    X_train, X_test, y_train, y_test = create_X_y_train_test_split(X=X, y=y, current_stock=ticker)

    X_test_index = np.arange(X_train.shape[0], X_train.shape[0]+X_test.shape[0])

    print(X_test_index)

    plt.plot(data.index[X_test_index], y_test, color='purple', label='Actual', linewidth="4.0")
    for model in models:
        plt.plot(data.index[X_test_index], stock_predictions[ticker][model+" Prediction"], label=model)
    plt.title(f'Actual vs Predicted, {ticker}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()

    #plt.savefig("actual_predicted_plots/"+ticker+".png")
    plt.show()

In [ ]:
# CORRELATION PLOT BETWEEN THE DIFFERENT STOCKS
adj_close_prices = pd.DataFrame({i: j[main_col] for i, j in stock_data.items()})

corr = adj_close_prices.corr()

sns.heatmap(corr, cmap="coolwarm", annot=True)

plt.title("Correlation between stocks")

#plt.savefig("correlation_plots/correlation_stocks.png")
plt.show()

# Saving the data

In [37]:
for ticker in stocks_list:
    print(ticker)
    display(metrics_output[ticker])
    #metrics_output[ticker].to_csv("saved_metrics/stock_"+ticker+".csv")
    #stock_data[ticker].to_pickle("saved_data_pickle/stock_"+ticker+".pkl")
    predicted_stock_data[ticker].to_csv("saved_predictions/_"+ticker+".csv")

EQNR.OL


,LR Model Train,LR Model Test,DTR Model Train,DTR Model Test,MLP Model Train,MLP Model Test,XGBoost Model Train,XGBoost Model Test,XGBoost_LR Model Train,XGBoost_LR Model Test,ADA Model Train,ADA Model Test,GBR Model Train,GBR Model Test,Bagging Model Train,Bagging Model Test,StackedRegressor Model Train,StackedRegressor Model Test
R^2:,0.996985,0.990769,1.0,-0.214380,0.991094,0.964790,0.999925,-0.266899,0.967126,0.734653,0.990928,-0.270874,0.998195,-0.232131,0.999012,-0.268687,0.996932,0.987479
MAE:,0.743181,1.633022,0.0,26.163471,1.231936,2.508528,0.127656,26.172964,2.322553,7.160678,1.446361,26.416699,0.615402,25.669145,0.376656,26.417137,0.744425,2.332848
MSE:,1.062961,5.990698,0.0,2973.662219,2.620583,12.762969,0.046184,3030.749249,8.299787,81.274224,3.599389,3000.766230,0.724019,2921.262496,0.333223,3002.072999,1.053638,16.429672
RMSE:,1.001151,2.077144,0.0,30.677009,1.609779,3.207665,0.177935,30.737324,2.845208,8.198133,1.813293,31.063832,0.797773,30.337502,0.561429,31.055130,1.000669,2.880701
MAPE:,0.006715,0.009519,0.0,0.099931,0.011304,0.016034,0.001123,0.097844,0.021952,0.044400,0.013135,0.100877,0.005528,0.096896,0.003430,0.102535,0.006736,0.011704


DNB.OL


,LR Model Train,LR Model Test,DTR Model Train,DTR Model Test,MLP Model Train,MLP Model Test,XGBoost Model Train,XGBoost Model Test,XGBoost_LR Model Train,XGBoost_LR Model Test,ADA Model Train,ADA Model Test,GBR Model Train,GBR Model Test,Bagging Model Train,Bagging Model Test,StackedRegressor Model Train,StackedRegressor Model Test
R^2:,0.998002,0.983546,1.0,-0.856536,0.992907,0.945687,0.999953,-0.871664,0.980911,0.866086,0.993257,-1.434293,0.998953,-0.786255,0.999328,-0.802833,0.998035,0.983394
MAE:,0.509645,0.868352,0.0,9.359718,0.816137,1.508948,0.090386,9.347225,1.391779,3.066436,1.062995,10.489471,0.408647,9.017193,0.259506,9.089574,0.503361,0.864117
MSE:,0.493871,1.428907,0.0,174.136664,1.140976,5.585848,0.021377,176.014437,3.226879,15.043678,1.970143,219.629367,0.319329,167.390655,0.175546,168.103330,0.479535,1.435764
RMSE:,0.686201,1.133516,0.0,12.278676,1.065704,2.137921,0.124787,12.418069,1.791846,3.646611,1.334923,13.430718,0.535573,12.013536,0.403432,12.067085,0.677175,1.136575
MAPE:,0.007209,0.008043,0.0,0.073903,0.012102,0.016030,0.001229,0.073846,0.022169,0.030361,0.015285,0.080857,0.005683,0.070377,0.003664,0.071104,0.007145,0.008027


TEL.OL


...

,LR Model Train,LR Model Test,DTR Model Train,DTR Model Test,MLP Model Train,MLP Model Test,XGBoost Model Train,XGBoost Model Test,XGBoost_LR Model Train,XGBoost_LR Model Test,ADA Model Train,ADA Model Test,GBR Model Train,GBR Model Test,Bagging Model Train,Bagging Model Test,StackedRegressor Model Train,StackedRegressor Model Test
R^2:,0.998952,0.990506,1.0,-1.839472,0.988862,0.968156,0.999981,-1.231077,0.992089,0.922960,0.995671,-1.336703,0.999517,-1.140787,0.999649,-1.144605,0.998741,0.989415
MAE:,0.638998,1.288049,0.0,16.824719,1.133521,2.279128,0.119343,15.946646,1.399639,3.246322,1.669670,16.328259,0.510020,15.211037,0.306631,15.224386,0.654097,1.333954
MSE:,0.895393,3.204626,0.0,544.750930,2.635649,9.303142,0.042872,517.692688,3.412336,17.106117,5.771571,534.368429,0.585606,482.974193,0.245824,492.101659,0.916591,3.349062
RMSE:,0.881441,1.688602,0.0,19.943286,1.536275,2.898530,0.165680,19.220786,1.789781,3.981474,2.114815,19.584958,0.686433,18.476414,0.469815,18.482437,0.901027,1.750849
MAPE:,0.008083,0.008576,0.0,0.123415,0.015487,0.017040,0.001468,0.113480,0.021246,0.026051,0.025576,0.117837,0.006153,0.110186,0.003812,0.110084,0.008363,0.009041


CL=F


,LR Model Train,LR Model Test,DTR Model Train,DTR Model Test,MLP Model Train,MLP Model Test,XGBoost Model Train,XGBoost Model Test,XGBoost_LR Model Train,XGBoost_LR Model Test,ADA Model Train,ADA Model Test,GBR Model Train,GBR Model Test,Bagging Model Train,Bagging Model Test,StackedRegressor Model Train,StackedRegressor Model Test
R^2:,0.996575,0.987903,1.0,0.129621,0.982882,0.979212,0.999959,0.280338,0.963190,0.462852,0.991886,0.180349,0.998569,0.233707,0.998896,0.192584,0.996882,0.986949
MAE:,0.575141,0.856079,0.0,8.464470,0.840873,1.237754,0.089111,7.554537,1.615894,4.674823,1.060319,8.380880,0.448366,7.717834,0.285155,8.076862,0.545760,0.960926
MSE:,0.545862,1.554690,0.0,297.042538,1.221928,3.165144,0.019427,260.902947,4.370454,47.124390,1.725674,283.861814,0.326660,280.745720,0.158959,284.523953,0.492240,1.966455
RMSE:,0.738008,1.153589,0.0,10.027757,1.056359,1.580250,0.122525,8.922371,2.052797,5.862376,1.302739,9.903212,0.563966,9.143775,0.398350,9.549923,0.700607,1.260132
MAPE:,0.009054,0.015343,0.0,0.184940,0.012319,0.021960,0.001464,0.167498,0.026410,0.099569,0.016861,0.181413,0.007076,0.171894,0.004492,0.177457,0.008556,0.017739


OSEBX.OL


,LR Model Train,LR Model Test,DTR Model Train,DTR Model Test,MLP Model Train,MLP Model Test,XGBoost Model Train,XGBoost Model Test,XGBoost_LR Model Train,XGBoost_LR Model Test,ADA Model Train,ADA Model Test,GBR Model Train,GBR Model Test,Bagging Model Train,Bagging Model Test,StackedRegressor Model Train,StackedRegressor Model Test
R^2:,0.994848,0.979712,1.0,0.125176,0.820841,0.776224,0.999962,0.156037,0.911728,0.323643,0.988226,-0.250857,0.997741,0.129419,0.998428,0.167747,0.995239,0.979147
MAE:,3.163232,4.788909,0.0,44.864994,10.663121,14.103808,0.439403,43.684256,14.501346,41.679637,6.173987,53.017135,2.454449,43.827614,1.562857,43.333654,3.092078,4.799669
MSE:,17.624453,43.234834,0.0,5719.018999,240.674259,396.297767,0.548877,5953.288481,321.697051,2531.948640,64.197117,6927.159022,10.599083,5850.139278,5.710568,5591.893622,17.114569,44.461639
RMSE:,4.162509,6.329614,0.0,59.851689,12.715524,17.029071,0.607344,59.713119,17.702334,45.634958,7.667849,68.623445,3.145412,59.989699,2.364190,58.848024,4.088779,6.426422
MAPE:,0.004804,0.005503,0.0,0.047953,0.017121,0.019347,0.000632,0.046523,0.022299,0.047157,0.009242,0.055653,0.003675,0.046529,0.002368,0.046238,0.004689,0.005541
